In [15]:
import numpy as np
import pandas as pd
import re, nltk, gensim #spacy

/Users/amberwtchang/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/amberwtchang/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [11]:
# setting up our imports
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger

In [49]:
# open title data
data_path = "dataset/"
df_corona = pd.read_json(data_path + 'corona_dtm.json')
df_corona

,Pmid,Year,Title,Abstract,Keywords,norm_tiabs,Month,Date
0,32857764,2020,"[Translesion synthesis by AMV, HIV, and MMLVre...",[Inosine is ubiquitous and essential in many b...,[],"Translesion synthesis by AMV, HIV, and MMLVrev...",8,1596240000000
1,32849762,2020,[LROD: An Overlap Detection Algorithm for Long...,[Third-generation sequencing technologies can ...,"[[alignment], [k-mer distribution], [long read...",LROD: An Overlap Detection Algorithm for Long ...,8,1596240000000
2,32849463,2020,[Analytical Performance Validation of Next-Gen...,[Next-generation sequencing (NGS) enables clin...,"[[antimicrobial resistance], [human pathogens]...",Analytical Performance Validation of Next-Gene...,8,1596240000000
3,32843552,2020,[A Genome-Based Model to Predict the Virulence...,[Variation in the genome of Pseudomonas aerugi...,"[[], [genome analysis], [machine learning], [m...",A Genome-Based Model to Predict the Virulence ...,8,1596240000000
4,32838395,2020,[The world should establish an early warning s...,[With the emergence of several new epidemics o...,[],The world should establish an early warning sy...,8,1596240000000
...,...,...,...,...,...,...,...,...
67963,28309716,2020,[Shallow water meiobenthos of the bermuda plat...,[The distribution and abundance of subtidal me...,[],Shallow water meiobenthos of the bermuda platf...,4,1585699200000
67964,5504579,1971,[Fats in fresh water crustaceans. I. Fatty aci...,[],[],Fats in fresh water crustaceans. I. Fatty acid...,5,41904000000
67965,28304606,2019,[[Patterns of differentiation of medusae buds ...,[1. The normal development of medusa ofPodocor...,[],[Patterns of differentiation of medusae buds a...,11,1572566400000
67966,14325028,1996,[THE ADULT AND LARVAL MORPHOLOGY AND LIFE HIST...,[],"[[ANATOMY], [BRYOZOA], [EMBRYOLOGY], [EXPERIME...",THE ADULT AND LARVAL MORPHOLOGY AND LIFE HISTO...,12,849398400000


In [50]:
# make date column
date = []
id = 0

for month in df_corona["Month"]:
    string_date = str(month) + "/" + str(df_dtm["Year"][id])
    date.append(string_date)
    id += 1
    
df_corona["Date"] = date
df_corona

,Pmid,Year,Title,Abstract,Keywords,norm_tiabs,Month,Date
0,32857764,2020,"[Translesion synthesis by AMV, HIV, and MMLVre...",[Inosine is ubiquitous and essential in many b...,[],"Translesion synthesis by AMV, HIV, and MMLVrev...",8,8/2020
1,32849762,2020,[LROD: An Overlap Detection Algorithm for Long...,[Third-generation sequencing technologies can ...,"[[alignment], [k-mer distribution], [long read...",LROD: An Overlap Detection Algorithm for Long ...,8,8/2020
2,32849463,2020,[Analytical Performance Validation of Next-Gen...,[Next-generation sequencing (NGS) enables clin...,"[[antimicrobial resistance], [human pathogens]...",Analytical Performance Validation of Next-Gene...,8,8/2020
3,32843552,2020,[A Genome-Based Model to Predict the Virulence...,[Variation in the genome of Pseudomonas aerugi...,"[[], [genome analysis], [machine learning], [m...",A Genome-Based Model to Predict the Virulence ...,8,8/2020
4,32838395,2020,[The world should establish an early warning s...,[With the emergence of several new epidemics o...,[],The world should establish an early warning sy...,8,8/2020
...,...,...,...,...,...,...,...,...
67963,28309716,2020,[Shallow water meiobenthos of the bermuda plat...,[The distribution and abundance of subtidal me...,[],Shallow water meiobenthos of the bermuda platf...,4,4/2020
67964,5504579,1971,[Fats in fresh water crustaceans. I. Fatty aci...,[],[],Fats in fresh water crustaceans. I. Fatty acid...,5,5/1971
67965,28304606,2019,[[Patterns of differentiation of medusae buds ...,[1. The normal development of medusa ofPodocor...,[],[Patterns of differentiation of medusae buds a...,11,11/2019
67966,14325028,1996,[THE ADULT AND LARVAL MORPHOLOGY AND LIFE HIST...,[],"[[ANATOMY], [BRYOZOA], [EMBRYOLOGY], [EXPERIME...",THE ADULT AND LARVAL MORPHOLOGY AND LIFE HISTO...,12,12/1996


In [54]:
df_corona['Date'] =pd.to_datetime(df_corona.Date)
df_corona = df_corona.sort_values(by='Date') # This now sorts in date order

In [60]:
df_corona = df_corona.reset_index(drop=True)
df_corona

,Pmid,Year,Title,Abstract,Keywords,norm_tiabs,Month,Date
0,4310292,1969,[[Peculiarities of fine structure and virus re...,[],[],[Peculiarities of fine structure and virus rep...,12,1969-12-01
1,5630129,1969,[[On infectious gastroenteritis of swine].],[],[],[On infectious gastroenteritis of swine].,12,1969-12-01
2,5358406,1970,[Observations on the experimental infection an...,[],[],Observations on the experimental infection and...,1,1970-01-01
3,5390019,1970,[[Identification of transmissible porcine gast...,[],[],[Identification of transmissible porcine gastr...,1,1970-01-01
4,5391878,1970,[[Identification of transmissible gastroenteri...,[],[],[Identification of transmissible gastroenterit...,3,1970-03-01
...,...,...,...,...,...,...,...,...
67963,32700814,2020,[COVID-19 infection in patients with Sézary sy...,[],[],COVID-19 infection in patients with Sézary syn...,8,2020-08-01
67964,32700813,2020,[Disseminated Intravascular Coagulation: A Dev...,[Disseminated intravascular coagulation (DIC) ...,"[[COVID-19], [bullae], [disseminated intravasc...",Disseminated Intravascular Coagulation: A Deva...,8,2020-08-01
67965,32700795,2020,[Targeting SARS-CoV-2 RBD interface: a supervi...,[Coronavirus Disease 2019 (COVID-19) has sprea...,"[[COVID-19], [Molecular dynamics], [Pharmacoph...",Targeting SARS-CoV-2 RBD interface: a supervis...,8,2020-08-01
67966,32700764,2020,[Screening with HRCT chest and PCR testing for...,[],[],Screening with HRCT chest and PCR testing for ...,8,2020-08-01


In [79]:
# calculate time slice for period of 10 years
peri10 = [0, 0, 0, 0, 0]

for date in df_corona["Date"]:
    if date < pd.Timestamp('1981-01-01T00'):
        peri10[0] += 1
    if date >= pd.Timestamp('1981-01-01T00') and date < pd.Timestamp('1991-01-01T00'):
        peri10[1] += 1
    if date >= pd.Timestamp('1991-01-01T00') and date < pd.Timestamp('2001-01-01T00'):
        peri10[2] += 1
    if date >= pd.Timestamp('2001-01-01T00') and date < pd.Timestamp('2011-01-01T00'):
        peri10[3] += 1
    if date >= pd.Timestamp('2011-01-01T00'):
        peri10[4] += 1
        
print(peri10)
#peri10[0]+peri10[1]+peri10[2]+peri10[3]+peri10[4]

[329, 657, 1557, 7839, 57586]


In [81]:
# calculate time slice for period of 10 years
peri05 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for date in df_corona["Date"]:
    if date < pd.Timestamp('1976-01-01T00'):
        peri05[0] += 1
    if date >= pd.Timestamp('1976-01-01T00') and date < pd.Timestamp('1981-01-01T00'):
        peri05[1] += 1
    if date >= pd.Timestamp('1981-01-01T00') and date < pd.Timestamp('1986-01-01T00'):
        peri05[2] += 1
    if date >= pd.Timestamp('1986-01-01T00') and date < pd.Timestamp('1991-01-01T00'):
        peri05[3] += 1
    if date >= pd.Timestamp('1991-01-01T00') and date < pd.Timestamp('1996-01-01T00'):
        peri05[4] += 1
    if date >= pd.Timestamp('1996-01-01T00') and date < pd.Timestamp('2001-01-01T00'):
        peri05[5] += 1
    if date >= pd.Timestamp('2001-01-01T00') and date < pd.Timestamp('2006-01-01T00'):
        peri05[6] += 1
    if date >= pd.Timestamp('2006-01-01T00') and date < pd.Timestamp('2011-01-01T00'):
        peri05[7] += 1
    if date >= pd.Timestamp('2011-01-01T00') and date < pd.Timestamp('2016-01-01T00'):
        peri05[8] += 1
    if date >= pd.Timestamp('2016-01-01T00'):
        peri05[9] += 1
        
print(peri05)

[152, 177, 255, 402, 668, 889, 4128, 3711, 3390, 54196]


In [84]:
# calculate time slice for period of 10 years
peri2019 = [0, 0, 0, 0, 0, 0, 0, 0, 0]

for date in df_corona["Date"]:
    if date < pd.Timestamp('2020-01-01T00'):
        peri2019[0] += 1
    if date >= pd.Timestamp('2020-01-01T00') and date < pd.Timestamp('2020-02-01T00'):
        peri2019[1] += 1
    if date >= pd.Timestamp('2020-02-01T00') and date < pd.Timestamp('2020-03-01T00'):
        peri2019[2] += 1
    if date >= pd.Timestamp('2020-03-01T00') and date < pd.Timestamp('2020-04-01T00'):
        peri2019[3] += 1
    if date >= pd.Timestamp('2020-04-01T00') and date < pd.Timestamp('2020-05-01T00'):
        peri2019[4] += 1
    if date >= pd.Timestamp('2020-05-01T00') and date < pd.Timestamp('2020-06-01T00'):
        peri2019[5] += 1
    if date >= pd.Timestamp('2020-06-01T00') and date < pd.Timestamp('2020-07-01T00'):
        peri2019[6] += 1
    if date >= pd.Timestamp('2020-07-01T00') and date < pd.Timestamp('2020-08-01T00'):
        peri2019[7] += 1
    if date >= pd.Timestamp('2020-08-01T00'):
        peri2019[8] += 1
        
print(peri2019)
sum(peri2019)

[17853, 95, 276, 874, 2798, 4660, 10000, 12434, 18978]


67968

In [61]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amberwtchang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
# Write a function to perform lemmatize and stem preprocessing steps on the data set.
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # lemmatize, pos
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result
lemmatize_stemming("rained")

'rain'

In [63]:
data_lemmatized = []
for d in df_corona["norm_tiabs"]:
    lemma_doc = preprocess(d)
    data_lemmatized.append(lemma_doc)

In [64]:
# store dtm_lemmatized list
import pickle

with open("dataset/dtm_lemmatized", "wb") as fp:   #Pickling
    pickle.dump(data_lemmatized, fp)


In [65]:
# open pickle file
with open("dataset/dtm_lemmatized", "rb") as fp:   # Unpickling
    data_lemmatized = pickle.load(fp)
len(data_lemmatized)

67968

In [66]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [67]:
# store dict_dtm
with open("dataset/dict_dtm", "wb") as fp:   #Pickling
    pickle.dump(id2word, fp)

In [68]:
# store corpus_dtm
with open("dataset/corpus_dtm", "wb") as fp:   #Pickling
    pickle.dump(corpus, fp)


In [86]:
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel

peri10 = [329, 657, 1557, 7839, 57586]
ldaseq = LdaSeqModel(corpus=corpus, time_slice=peri10, num_topics=15, chunksize=100)

/Users/amberwtchang/opt/anaconda3/lib/python3.7/site-packages/gensim/models/ldaseqmodel.py:1474: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))


KeyboardInterrupt: 

In [ ]:
# save model
from gensim.test.utils import datapath

temp_file = datapath("model")
ldaseq.save(temp_file)

In [ ]:
# Load a potentially pre-trained model from disk.
ldaseq = LdaSeqModel.load(temp_file)

In [85]:
# loading our corpus and dictionary
try:
    dictionary = Dictionary.load('dataset/dict_dtm')
except FileNotFoundError as e:
    raise ValueError("SKIP: Please find dataset/dict_dtm dictionary=id2vec")

    
corpus = bleicorpus.BleiCorpus('dataset/corpus_dtm')
# it's very important that your corpus is saved in order of your time-slices!

time_slice = [329, 657, 1557, 7839, 57586]

OSError: BleiCorpus: could not find vocabulary file

In [9]:
ldaseq

In [10]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pyLDAvis.enable_notebook()
# This is apply on sklearn best lda model (select from gesim result), topic = 7
panel = pyLDAvis.sklearn.prepare(ldaseq, data_vectorized, vectorizer, mds='tsne')
panel

In [3]:
from gensim.test.utils import datapath

In [4]:
temp_file = datapath("model")

In [5]:
ldaseq.save(temp_file)

In [6]:
# Load a potentially pre-trained model from disk.
ldaseq = LdaSeqModel.load(temp_file)

In [7]:
doc = common_corpus[1]

In [8]:
embedding = ldaseq[doc]